In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import file_classes
from file_classes import SpatialLightModulator, PhotostimulationDevice, ImagingPlane, HolographicPattern, PhotostimulationSeries

In [2]:
from datetime import datetime
from dateutil.tz import tzlocal

from pynwb import NWBFile

nwbfile = NWBFile(
    'my first synthetic recording',
    'EXAMPLE_ID',
    datetime.now(tzlocal()),
)

import numpy as np
slm = SpatialLightModulator(
    name="slm",
    dimensions = [1, 2]
)

dev = PhotostimulationDevice(name="device", description="photostim_device", type='LED', wavelength=320, slm=slm)

In [3]:
from datetime import datetime
from dateutil.tz import tzlocal

from pynwb import NWBFile

nwbfile = NWBFile(
    'my first synthetic recording',
    'EXAMPLE_ID',
    datetime.now(tzlocal()),
)

import numpy as np
slm = SpatialLightModulator(
    name="slm",
    dimensions = [1, 2, 3],#[32, 10],
)

dev = PhotostimulationDevice(name="device", description="photostim_device", type='LED', wavelength=320.)
nwbfile.add_device(dev)

dev = PhotostimulationDevice(name="device1")
nwbfile.add_device(dev)

imaging_plane = ImagingPlane(name="imaging_plane",  device=dev)

In [15]:
# hp = HolographicPattern(name='HolographicPattern', description='HolographicPattern')
import numpy as np

image_mask = np.zeros((100, 100))

# randomly generate example image masks
x = np.random.randint(0, 95)
y = np.random.randint(0, 95)
image_mask[x:x + 5, y:y + 5] = 1
hp = HolographicPattern(name='HolographicPattern',  mask_roi= image_mask)

In [16]:
from file_classes import PhotostimulationSeries
ser = PhotostimulationSeries(name="photosim series", pattern=hp, unit='SIunit',  field_of_view=1, imaging_plane=imaging_plane)

ser = PhotostimulationSeries(name="photosim series", pattern=hp, unit='SIunit',  data=[1, 1, 1], timestamps=[0.5, 1, 2], imaging_plane=imaging_plane)

ser.add_interval(10, 20)

nwbfile.add_stimulus(ser)

In [17]:
from file_classes import PhotostimulationSeries
ser = PhotostimulationSeries(name="photosim series", pattern=hp, unit='SIunit',  field_of_view=1, format='series', data=[0, 1, 0], rate=10., imaging_plane=imaging_plane)

/Users/harriscaw/PycharmProjects/nwb-photostim/file_classes.py:202: UserWarning: if 'format' is 'series', 'stimulus_duration' should be specified
  warnings.warn("if 'format' is 'series', 'stimulus_duration' should be specified")


In [12]:
x = np.random.randint(0, 95)
y = np.random.randint(0, 95)

pixel_roi = []
for ix in range(x, x + 5):
    for iy in range(y, y + 5):
        pixel_roi.append((ix, iy, 1))

hp2 = HolographicPattern(name='HolographicPattern', pixel_roi=pixel_roi, stimulation_diameter=5, dimension=[100, 100])
ser = PhotostimulationSeries(name="photosim series2", pattern=hp2, unit='SIunit',  imaging_plane=imaging_plane)
# nwbfile.add_stimulus(ser)

In [23]:
from pynwb import NWBHDF5IO

f_name = 'holographic_photosim_example.nwb'
io = NWBHDF5IO(f_name, mode='w')
io.write(slm)
io.close()

In [3]:
from pynwb import NWBHDF5IO

with NWBHDF5IO("basics_tutorial.nwb", "w") as io:
    io.write(dev)
#

In [4]:
with NWBHDF5IO("basics_tutorial.nwb", "r", load_namespaces=True) as io:
    read_nwbfile = io.read()

In [20]:
from pynwb import NWBHDF5IO
dev = PhotostimulationDevice(name="device", description="photostim_device", type='LED', wavelength=320., slm=slm)

dev = PhotostimulationDevice(name="device", description="photostim_device", type='LED')

dev = PhotostimulationDevice(name="device", description="photostim_device", type='LED', wavelength=320., slm=slm)

f = "basics_tutorial.nwb"
io = NWBHDF5IO(f, 'w')
io.write(slm)
io.write(imaging_plane.device)
io.write(imaging_plane)
io.close()

/Users/harriscaw/opt/anaconda3/envs/NWB/lib/python3.10/site-packages/hdmf/build/objectmapper.py:660: MissingRequiredBuildWarning: ImagingPlane 'imaging_plane' is missing required value for attribute 'pulse_rate'.
  warnings.warn(msg, MissingRequiredBuildWarning)
/Users/harriscaw/opt/anaconda3/envs/NWB/lib/python3.10/site-packages/hdmf/build/objectmapper.py:660: MissingRequiredBuildWarning: ImagingPlane 'imaging_plane' is missing required value for attribute 'power'.
  warnings.warn(msg, MissingRequiredBuildWarning)
/Users/harriscaw/opt/anaconda3/envs/NWB/lib/python3.10/site-packages/hdmf/build/objectmapper.py:660: MissingRequiredBuildWarning: ImagingPlane 'imaging_plane' is missing required value for attribute 'peak_pulse_power'.
  warnings.warn(msg, MissingRequiredBuildWarning)
/Users/harriscaw/opt/anaconda3/envs/NWB/lib/python3.10/site-packages/hdmf/build/objectmapper.py:660: MissingRequiredBuildWarning: ImagingPlane 'imaging_plane' is missing required value for attribute 'opsin'.
  

In [22]:
io.close()